In [17]:
# *****************************************************************************
#  Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
#  Redistribution and use in source and binary forms, with or without
#  modification, are permitted provided that the following conditions are met:
#      * Redistributions of source code must retain the above copyright
#        notice, this list of conditions and the following disclaimer.
#      * Redistributions in binary form must reproduce the above copyright
#        notice, this list of conditions and the following disclaimer in the
#        documentation and/or other materials provided with the distribution.
#      * Neither the name of the NVIDIA CORPORATION nor the
#        names of its contributors may be used to endorse or promote products
#        derived from this software without specific prior written permission.
#
#  THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
#  ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
#  WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
#  DISCLAIMED. IN NO EVENT SHALL NVIDIA CORPORATION BE LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
#  (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
#  LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
#  ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
#  (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
#  SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# *****************************************************************************
import argparse
import json
import os
import time
import torch
import numpy as np

#=====START: ADDED FOR DISTRIBUTED======
from distributed import init_distributed, apply_gradient_allreduce, reduce_tensor
from torch.utils.data.distributed import DistributedSampler
#=====END:   ADDED FOR DISTRIBUTED======

from torch.utils.data import DataLoader
from wavenet import WaveNet
from mel2samp_onehot import Mel2SampOnehot
from utils import to_gpu
from domain_confusion import DomainCNN

## Get Data

In [19]:
from music_data import MusicDataset

root_dir = "../music_data"
artists = ["Perlman", "Milstein"]
data = []
for artist in artists:
    d = MusicDataset("{}/{}".format(root_dir, artist))
    data.append(d)
data

Itzhak Perlman, Bach Sonata No.2 in A minor BWV 1003.mp3
successfully loaded 62234 1-second (22050 sample) clip(s) from Itzhak Perlman, Bach Sonata No.2 in A minor BWV 1003.mp3


MemoryError: 

In [18]:
data_len = min(len(d) for d in data)
stacked_data = np.hstack(tuple(d.data[0:data_len] for d in data))

dataloader = torch.utils.data.DataLoader(stacked_data, batch_size=10, shuffle=True, num_workers=1)


    

MemoryError: 

In [16]:
stacked_data.shape

(38315, 44100)

In [ ]:
class DomainCNN(torch.nn.Module):
    # input is vector in R^64

    def __init__(self, domains):
        super(DomainCNN, self).__init__()

        # 3 1D convolution layers
        self.conv1 = torch.nn.Conv1d(1, 32, kernel_size=5)
        self.pool1 = torch.nn.MaxPool1d(kernel_size=2)
        self.conv2 = torch.nn.Conv1d(32, 16, kernel_size=5, stride=2)
        self.pool2 = torch.nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv3 = torch.nn.Conv1d(16, 8, kernel_size=2, stride=2)
        self.pool3 = torch.nn.MaxPool1d(kernel_size=2, stride=1)

        # last layer projects vectors to dimension k
        # average the vectors to obtain a single vector of dim k
        self.fc1 = torch.nn.Linear(8*2, domains)

    def forward(self, x):
        x = F.elu(self.conv1(x))
        x = self.pool1(x)
        x = F.elu(self.conv2(x))
        x = self.pool2(x)
        x = F.elu(self.conv3(x))
        x = self.pool3(x)
        # reshape
        x = x.view(-1, 8*2)
        m = torch.nn.Softmax(1)
        x = m(self.fc1(x))
        return x


### Configure encoders and decoders

In [ ]:
encoder_config = {
         "n_in_channels": 1,
         "n_layers": 10,
         "max_dilation": 128,
         "n_residual_channels": 64,
         "n_skip_channels": 256,
         "n_out_channels": 128,
         "n_cond_channels": 80,
         "upsamp_window": 800,
         "upsamp_stride": 200
     }
decoder_config =  {
         "n_in_channels": 128,
         "n_layers": 10,
         "max_dilation": 128,
         "n_residual_channels": 64,
         "n_skip_channels": 256,
         "n_out_channels": 256,
         "n_cond_channels": 80,
         "upsamp_window": 800,
         "upsamp_stride": 200
     }

train_config = {
         "output_directory": "checkpoints",
         "epochs": 100000,
         "learning_rate": 1e-3,
         "iters_per_checkpoint": 1000,
         "batch_size": 8,
         "seed": 1234,
         "checkpoint_path": ""
     },

# distributed config if you have multiple GPUs. otherwise leave alone
"dist_config": {
         "dist_backend": "nccl",
         "dist_url": "tcp://localhost:54321"
     },

In [ ]:
# One autoencoder per domain
class MusicAutoEncoder(torch.nn.Module):
    def __init__(self, domains, domain_cnn):
        """param domains: int specifying number of domains
           param domain_cnn: a Domain Confusion CNN"""
        super(MusicAutoEncoder, self).__init__()
        self.domains = domains
        self.encoder = WaveNet(**encoder_config).cuda()
        self.decoders = [WaveNet(**decoder_config).cuda() for k in range(domains)]
        self.domain_cnn = domain_cnn

    def forward(self, forward_input):
        encoder_out = self.encoder(forward_input)
        decoders_out = [self.decoders[domain_num](encoder_out) for domain_num in range(self.domains)]
        return encoder_out, decoders_out


In [ ]:

class AutoEncoderLoss(torch.nn.Module):
    def __init__(self, num_classes):
        super(AutoEncoderLoss, self).__init__()
        self.num_classes = num_classes

    def forward(self, inputs, domain_cnn_output, decoder_outputs, lambda=0.5):
        """
        inputs = sample? s_j? are batch size by k
        inputs are output of encoder, and k decoders
        targets are batch by sample
        torch CrossEntropyLoss needs
            input = batch * samples by num_classes
            targets = batch * samples
        """
        targets = targets.view(-1)
        inputs = inputs.transpose(1, 2)
        inputs = inputs.contiguous()
        inputs = inputs.view(-1, self.num_classes)
        loss = 0
        for j in range(decoder_outputs.shape[1]):
            j_loss = 0
            for sample in inputs[:,j]:
                lhs = torch.nn.CrossEntropyLoss(decoder_outputs[j], sample)
                rhs = lambda * torch.nn.CrossEntropyLoss(domain_cnn_output, j)
                j_loss += (lhs.sum() / lhs.shape[1]) - (rhs.sum() / rhs.shape[1])
            loss += j_loss
        return loss



In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer):
    assert os.path.isfile(checkpoint_path)
    checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
    iteration = checkpoint_dict['iteration']
    optimizer.load_state_dict(checkpoint_dict['optimizer'])
    model_for_loading = checkpoint_dict['model']
    model.load_state_dict(model_for_loading.state_dict())
    print("Loaded checkpoint '{}' (iteration {})" .format(
          checkpoint_path, iteration))
    return model, optimizer, iteration

def save_checkpoint(model, config, optimizer, learning_rate, iteration, filepath):
    print("Saving model and optimizer state at iteration {} to {}".format(
          iteration, filepath))
    model_for_saving = WaveNet(**config).cuda()
    model_for_saving.load_state_dict(model.state_dict())
    torch.save({'model': model_for_saving,
                'iteration': iteration,
                'optimizer': optimizer.state_dict(),
                'learning_rate': learning_rate}, filepath)

def train(rank, group_name, output_directory, epochs, learning_rate,
          iters_per_checkpoint, batch_size, seed, checkpoint_path, num_domains, domain_confusion_model, train_loader, num_gpus=1):
    """param num_domains: number of music domains
       param domain_confusion_model: domain confusion network model"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #=====START: ADDED FOR DISTRIBUTED======
    if num_gpus > 1:
        init_distributed(rank, num_gpus, group_name, **dist_config)
    #=====END:   ADDED FOR DISTRIBUTED======

    ae_criterion = AutoEncoderLoss() # autoencoder
    confusion_criterion = torch.nn.CrossEntropyLoss() # domain confusion
    model = MusicAutoEncoder().cuda()

    #=====START: ADDED FOR DISTRIBUTED======
    if num_gpus > 1:
        model = apply_gradient_allreduce(model)
    #=====END:   ADDED FOR DISTRIBUTED======

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Load checkpoint if one exists
    iteration = 0
    if checkpoint_path != "":
        model, optimizer, iteration = load_checkpoint(checkpoint_path, model,
                                                      optimizer)
        iteration += 1  # next iteration is iteration + 1

    # =====START: ADDED FOR DISTRIBUTED======
    train_sampler = DistributedSampler(trainset) if num_gpus > 1 else None
    # =====END:   ADDED FOR DISTRIBUTED======


    # Get shared output_directory ready
    if rank == 0:
        if not os.path.isdir(output_directory):
            os.makedirs(output_directory)
            os.chmod(output_directory, 0o775)
        print("output directory", output_directory)

    model.train()
    epoch_offset = max(0, int(iteration / len(train_loader)))
    # ================ MAIN TRAINING LOOP! ===================
    for epoch in range(epoch_offset, epochs):
        print("Epoch: {}".format(epoch))
        for i, batch in enumerate(train_loader):
            model.zero_grad()
            # TODO needs debugging from around here
            x, y = batch
            x = to_gpu(x).float()
            y = to_gpu(y)
            x = (x, y)  # auto-regressive takes outputs as inputs
            encoder_out, decoder_outs = model(x)
            domain_pred = domain_confusion_model(encoder_out)
            ae_loss = ae_criterion(x, domain_pred, decoder_outs) #autoencoder loss
            confusion_loss = ae_criterion(domain_pred, y)
            if num_gpus > 1:
                reduced_loss = reduce_tensor(loss.data, num_gpus)[0]
            else:
                reduced_loss = loss.data[0]
            ae_loss.backward()
            confusion_loss.backward()
            optimizer.step()

            print("{}:\t{:.9f}".format(iteration, reduced_loss))

            if (iteration % iters_per_checkpoint == 0):
                if rank == 0:
                    checkpoint_path = "{}/wavenet_{}".format(
                        output_directory, iteration)
                    save_checkpoint(model, optimizer, learning_rate, iteration,
                                    checkpoint_path)

            iteration += 1

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--config', type=str,
                        help='JSON file for configuration')
    parser.add_argument('-r', '--rank', type=int, default=0,
                        help='rank of process for distributed')
    parser.add_argument('-g', '--group_name', type=str, default='',
                        help='name of group for distributed')
    args = parser.parse_args()


    num_gpus = torch.cuda.device_count()
    if num_gpus > 1:
        if args.group_name == '':
            print("WARNING: Multiple GPUs detected but no distributed group set")
            print("Only running 1 GPU.  Use distributed.py for multiple GPUs")
            num_gpus = 1

    if num_gpus == 1 and args.rank != 0:
        raise Exception("Doing single GPU training on rank > 0")

    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = False
    train(num_gpus, args.rank, args.group_name, **train_config)